In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! pip install -q transformers

! nvidia-smi

Mounted at /content/drive
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 83.7 MB/s 
     |████████████████████████████████| 6.6 MB 75.1 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
Tue Jul 26 02:11:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      D

In [ ]:
%cd /content/drive/MyDrive/NLP/ENG/ai4code/src2

import easydict
import gc
import pandas as pd

from preprocessor import PairwisePreprocessor, _20CodeCellPreprocessor
from dataset import _20SampleDataset, PairwiseDataset, _20sample_data_setup, pairwise_data_setup
from train import pairwise_train_setup, train_setup
from util import pairwise_debug_setup, _20sample_debug_setup
from metrics import kendall_tau
from train import train

args = {
    'model_name_or_path': 'microsoft/graphcodebert-base',

    'input_path': '../input/',

    'train_path': './data/train.csv',
    'train_mark_path': './data/train_mark.csv',
    'train_features_path': './data/train_fts.json',

    'val_path': "./data/val.csv",
    'val_mark_path': './data/val_mark.csv',
    'val_features_path': './data/val_fts.json',

    'output_path': './output-graphcodebert-20sample-debug',

    'md_max_len': 64,
    'total_max_len': 512,
    'batch_size': 16,
    'accumulation_steps': 2,
    'epoch': 0,
    'epochs': 5,
    'n_workers': 8,
    'debug': True,
    'load_train': False,
    'max_lr': 3e-5,
    'min_lr': .3e-6,
    'kfold': True
}

args = easydict.EasyDict(args)

preprocessor = _20CodeCellPreprocessor(**vars(args))
train_df, val_df, train_df_mark, val_df_mark, train_fts, val_fts = preprocessor.run()

print('before debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

if args.debug:
    kfolds = _20sample_debug_setup(train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts)
    train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[0]

print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

del train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts, preprocessor
gc.collect()

df_orders = pd.read_csv(args.input_path + 'train_orders.csv',
                        index_col='id',
                        squeeze=True).str.split()

for i in range(5):
    train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[i]

    train_loader, val_loader = _20sample_data_setup(train_df_mark, val_df_mark, train_fts, val_fts, args)

    del train_df, train_df_mark, train_fts
    gc.collect()

    args.num_train_steps = args.epochs * len(train_loader) / args.accumulation_steps

    model, optimizer, scheduler, scaler = train_setup(args)
    model.cuda()

    train(model, train_loader, val_loader, optimizer, scheduler, scaler, val_df, df_orders, args)

    del model, optimizer, scheduler, scaler, val_df, val_df, val_fts, train_loader, val_loader
    gc.collect()

/content/drive/MyDrive/NLP/ENG/ai4code/src2
train_df, val_df are already exits
train_fts, val_fts are already exists
before debug (5740832, 8) (629814, 8) (1950118, 8) (629814, 8) 125292 13964
after debug (574117, 8) (63276, 8) (194488, 8) (63276, 8) 12529 1396


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/539 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

Epoch 1 Loss: 0.364 lr: [1.974496092143151e-08, 1.974496092143151e-08]:   0%|          | 2/12155 [00:04<5:48:31,  1.72s/it] 

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 Loss: 0.1766 lr: [2.526263829050207e-05, 2.526263829050207e-05]: 100%|██████████| 12155/12155 [38:05<00:00,  5.32it/s]
100%|██████████| 1367/1367 [01:25<00:00, 15.94it/s]
Preds score 0.7967970268261816
Epoch 2 Loss: 0.1291 lr: [1.8946329212583082e-05, 1.8946329212583082e-05]: 100%|██████████| 12155/12155 [38:01<00:00,  5.33it/s]
100%|██████████| 1367/1367 [01:25<00:00, 15.97it/s]
Preds score 0.8055823057802804
